# TCG Player Yugioh Cards Scraper
This will scrape the cards from TCG Player.

  1. Search cards by card name
  2. Grab latest sales data from card


In [3]:
# we can see the robots.txt
import requests
print(requests.get('https://tcgplayer.com/robots.txt').text)

User-agent: *

Crawl-Delay: 10

Allow: /

Disallow: /*?*seller=*

Sitemap: https://www.tcgplayer.com/sitemap/index.xml




In [4]:
# set this as the output directory
output_dir = '/Users/hammad/yugioh_outputs_20240811'

## Web Scaping Limits & Rules
Our goal is not only to maintain compliance, but also contribute to the servers that we scrape. The requirements in robots.txt must be adhered to. Here are some of the fields and what they mean

### User-agent: *
The User-agent field is located in every request and can indicate if the request is coming from a web browser, operating system, and so on. The wildcard parameter, *, means 'All', and in this case there are no restrictions in the User-agent.
### Crawl-Delay: 10
In seconds, it describes the amount of delay between requests. This is important as it makes sure web scrapers are not overloading the server. There are an incredible number of methods to scrape but some of the fastest ones can make over 1 million requests all at once. 
### Allow: /
This explains which parts of the web application or URL can be scraped. a '/' indicates the root directory which means that everything is allowed, pending understanding of what is disallowed.
### Disallow: /*?*seller=*
Combined with the allowed statement, this provides endpoints or parts of the URL that are not allowed. In this case, TCGPlayer doesn't allow requests that scrape seller information as a parameter indicated by the wildcards. Query parameters in the URL are specified as ?parameter_name=parameter_value . We will also respect seller privacy even when we have seller data from a page that is allowed.
### Sitemap: https://www.tcgplayer.com/sitemap/index.xml
This provides a "hardcoded" list of links and metadata that can understand what it's about.

In [6]:
from xml.dom.minidom import parseString
sitemap = parseString(requests.get('https://www.tcgplayer.com/sitemap/index.xml').content)

In [7]:
# get all yugioh sitemaps
yugioh_maps = []
for loc in sitemap.getElementsByTagName('loc') : # all TCGPlayer collections like Pokemon or Magic
    current_map = loc.childNodes[0].data
    if "yugioh" in current_map :
        yugioh_maps.append(current_map)

## TCGPlayer Yugioh Sitemaps
Not all sites have a sitemap defined, but understanding the structure of it, here are some conclusions,

- Each yugioh map XML file has the list of cards
- We're interested in the URL's that define "product" instead of other's like "categories" or "search"
- The links seem to be sorted by newest to oldest products
- The newest products may not have any sales


In [9]:
print(yugioh_maps)

['https://www.tcgplayer.com/sitemap/yugioh.0.xml', 'https://www.tcgplayer.com/sitemap/yugioh.1.xml']


## Advanced Web Scraping
If we're lucky, we can get all the data we need using the built in Python tools like **requests**. However, some web apps run Javascript that provide the necessary tools. In our case, there's some pertinent sales data that is loaded dynamically from TCG Player. We could break it down and make multiple simple requests but it would also multiply our processing time by the same factor. This is because the **robots.txt** asks to make 1 request per 10 seconds. The data gap between putting a URL into a modern web browser and using Python's simple requests can be filled by using advanced web scraping libraries.

This takes the "engine" from the web browsers, referred to as "web drivers", and allows us to process the data similarly to what we experience from it. Additionally, some web browsers use different engines. Here, we will install the library in Python known as **playwright** and a few web drivers for compatibility. Other libraries are available including **Selenium** but it requires manual web driver installion. This will half the scraping time because we would need to make 2 requests from a simple request and at more than 33,000 cards to scrape, the time savings can equal to days.

### References
https://playwright.dev/python/docs/intro

In [11]:
!pip install pytest-playwright

In [12]:
# install web drivers
!playwright install

In [13]:
import nest_asyncio
nest_asyncio.apply()

In [14]:
example_url = 'https://www.tcgplayer.com/product/285222/yugioh-2022-tin-of-the-pharaohs-gods-ghost-mourner-and-moonlit-chill'
sales_uri = 'https://mpapi.tcgplayer.com/v2/product/26720/latestsales?mpfev=2631'

In [15]:
from playwright.async_api import async_playwright
import json
import asyncio

async def tcgplayer_yugioh_sales(url, filename, timeout=5000):
    '''
    Using Playwright, we download the URL and pertinent requests associated with it.

    Parameters
    ----------
    url: string
        The URL to go to, could be from a sitemap
        
    output_file: string
        The outputfile name or path

    timeout: int
        Default is 5000 or 5 seconds
        The number of milliseconds to wait

    References
    ----------
    https://playwright.dev/python/docs/network#network-events
    https://playwright.dev/python/docs/api/class-route#route-continue
    '''
    p = await async_playwright().start()
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()

    # Define a callback function to handle intercepted requests
    async def handle_route(route):
        request = route.request
        #print(request.url)
        # example url is https://mpapi.tcgplayer.com/v2/product/26720/latestsales?mpfev=2631
        if 'latestsales' in request.url and request.method == 'POST':
            print(request.url, end='\r')
            
            # modify request data for things like condition, # of results, etc.
            data = json.loads(request.post_data)
            data['conditions'] = [] # gets all conditions
            await route.continue_(post_data = data)
            # we can fetch the data again with different parameters, but only once for now
            response = await route.fetch()
            
            # Save the response body to a file
            with open(f'{filename}_sales.json', 'w') as file:
                result = await response.body()
                # test to see if it can be loaded as json
                json_test = json.loads(result)
                file.write(json.dumps(json_test))
        else:
            await route.continue_()

    # Intercept requests and call the callback function
    await page.route("**/*", handle_route)

    await page.goto(url)  # Go to the page that makes the request

    # Wait for a short time to ensure the request is captured
    # Removing this likely will break it because there's so many requests
    await page.wait_for_timeout(timeout)
    html_content = await page.content()  # Get the page content as a string
        
    with open(f'{filename}.html', 'w', encoding='utf-8') as file:
        file.write(html_content)  # Write the content to the file

    await browser.close()
    return True

await tcgplayer_yugioh_sales(example_url, 'test')

True

In [16]:
urls = []
for map in yugioh_maps:
    print(map)
    sitemap = parseString(requests.get(map).content)
    for loc in sitemap.getElementsByTagName('loc') : # all TCGPlayer collections like Pokemon or Magic
        current_url = loc.childNodes[0].data
        if '/product/' in current_url : # only grab if its a product
            urls.append(current_url)
        

https://www.tcgplayer.com/sitemap/yugioh.0.xml

https://www.tcgplayer.com/sitemap/yugioh.1.xml


In [17]:
# parse and save outputs in time descending order
import os.path
import time

async def get_url(url, output_path, overwrite=False):
    # run the scraper here
    completed = False
    count = 0
    while (not completed):
        if not overwrite and os.path.isfile(f'{output_path}.html') :
            #print(f'Already done with {output_path}')
            return False
        elif count > 5 : # limit the number of retries
            break
        else :
            try:
                start_time = time.time()
                completed = await tcgplayer_yugioh_sales(url, output_path, 7000)
                # 10 sec timeout per robots.txt
                timeout = 10 - (time.time() - start_time)
                if timeout > 0 :
                    print(f'Sleeping for {int(timeout)} seconds.', end='\r')
                    time.sleep(timeout)
            except:
                print('exception occured, trying again')
                count += 1
                completed = False

    return completed

for index, url in enumerate(urls[::-1]) :
    product_id = url[len('https://www.tcgplayer.com/product/'):].split('/')[0]
    output_path = f'{output_dir}/{product_id}'
    result = await get_url(url, output_path)
    if result :
        print(f'CID {output_path.split('/')[-1]} done. {index+1} out of {len(urls)} ({round((index+1)/len(urls), 5) * 100}%){" "*50}', end='\r')
    

In [25]:
# run all above here 

## Example Outputs
The following cells of code will outline some example outputs

In [28]:
import os
output_filepaths = os.listdir(output_dir)
output_filepaths[:5]

['249048.html',
 '262318.html',
 '22100_sales.json',
 '213056_sales.json',
 '32877_sales.json']

In [30]:
import json
import pprint
with open(f'{output_dir}/{output_filepaths[2]}', 'r') as f :
    example = json.loads(f.read())
example

{'previousPage': '',
 'nextPage': '',
 'resultCount': 20,
 'totalResults': 20,
 'data': [{'condition': 'Lightly Played',
   'variant': 'Unlimited',
   'language': 'English',
   'quantity': 1,
   'title': 'Insect Soldiers of the Sky',
   'listingType': 'ListingWithoutPhotos',
   'customListingId': '0',
   'purchasePrice': 0.25,
   'shippingPrice': 1.27,
   'orderDate': '2024-08-18T20:14:42.113+00:00'},
  {'condition': 'Moderately Played',
   'variant': 'Unlimited',
   'language': 'English',
   'quantity': 1,
   'title': 'Insect Soldiers of the Sky',
   'listingType': 'ListingWithoutPhotos',
   'customListingId': '0',
   'purchasePrice': 0.04,
   'shippingPrice': 0.95,
   'orderDate': '2024-08-13T21:02:28.257+00:00'},
  {'condition': 'Near Mint',
   'variant': 'Unlimited',
   'language': 'English',
   'quantity': 5,
   'title': 'Insect Soldiers of the Sky',
   'listingType': 'ListingWithoutPhotos',
   'customListingId': '0',
   'purchasePrice': 0.11,
   'shippingPrice': 0.0,
   'orderDat

## Transform Data
Now that we have the data, we can extract the information and transform it.

### Card Data
The card data is from each webpage (e.g. 57114.html) which represents 1 card. This extracts metadata from the card. Note that full card details are meant to be joined from the TCG Complete Card Database because it contains official attack, defense, descriptions, and other information from the official Konami source.

After this, we will parse through the sales data of each card.

In [33]:
from bs4 import BeautifulSoup

def card_webpage_extract(contents):
    '''
    Given the HTML contents as a string, this function uses BeautifulSoup to parse relevant
    contents from the card.

    This process involved opening up the local HTML file in a web browser, clicking on and
    inspecting the relevant data, finding their HTML tag names and classes, and finally
    coding those names and classes to automatically parse their data.
    '''
    soup = BeautifulSoup(contents, 'html.parser')
    
    # card name is a title on the webpage with distinct class
    card_name = soup.find('h1', {'class': 'product-details__name'}).text.strip()
    
    # iterate through list of attributes and save relevant information
    card_attributes = soup.find('ul', {'class': 'product__item-details__attributes'})
    card_number = None # box sets and etc. don't have card numbers
    card_rarity = None # sets don't have rarity, for example
    if card_attributes :
        number_string = 'Number:'
        rarity_string = 'Rarity:'
        for attribute in card_attributes.findChildren():
            # some strings have the search string in them, so we check if it's already defined
            if not card_number and number_string in attribute.text:
                card_number = attribute.text.replace(number_string, '').strip()
            if not card_rarity and rarity_string in attribute.text:
                card_rarity = attribute.text.replace(rarity_string, '').strip()

    # get the card's 3 month change in price
    card_market = soup.find('div', {'class': 'charts-row'})
    card_price_3m_change = None # some cards don't have enough sales for this value
    if card_market:
        for market_info in card_market.findChildren() :
            # get the percent change
            if '%' in market_info.text :
                card_price_3m_change = market_info.text.strip()
                break

    def parse_price_points(section, label):
        '''
        Get card's marketpace data. Preprocessed example:
        ' Market Price $0.05 Most Recent Sale $0.06 Buylist Price: - ' +
        'Listed Median: - Current Quantity: 2314 Current Sellers: 212'
        '''
        return section.text.split(label)[-1].strip().split(' ')[0]
        
    card_marketplace = soup.find('section', {'class': 'price-guide__points'})
    if card_marketplace:
        card_price = parse_price_points(card_marketplace, 'Market Price')
        card_quantity = parse_price_points(card_marketplace, 'Current Quantity:')
        card_sellers = parse_price_points(card_marketplace, 'Current Sellers:')
    

    results = {
        'name': card_name,
        'number': card_number,
        'rarity': card_rarity,
        'price': card_price,
        'quantity': card_quantity,
        'quantity_sellers': card_sellers,
        'price_change_3m': card_price_3m_change
    }
    
    return results


In [39]:
# process cards.
# warning: running here again after redoing malformed may take long, use the designated cell
card_data = []
malformed_cards = []
for index, filepath in enumerate(output_filepaths) :
    if filepath.split('.')[-1] == 'html' :
        print(f'Processing {filepath} ({round(((index+1)/len(output_filepaths)), 4)*100}%){" "*50}', end='\r')
        # open HTML file
        full_path = f'{output_dir}/{filepath}'
        with open(full_path, 'r') as f:
            card_webpage = f.read()

        # extract and save
        try:
            card_extract = card_webpage_extract(card_webpage)
            card_extract.update({
                'index': filepath.split('.')[0],
                'price_asof': os.path.getmtime(full_path)
            })
        except Exception as e:
            print(f'\n{filepath}\n{e}')
            malformed_cards.append((filepath, e))
        card_data.append(card_extract)

In [41]:
len(card_data)

39899

In [43]:
# only expected value from errors is {"'NoneType' object has no attribute 'text'"}
set([str(card[1]) for card in malformed_cards])

set()

In [45]:
malformed_cards

[]

In [47]:
# get the urls for the malformed cards
malformed_urls = []
for redo in malformed_cards:
    cid = redo[0].split('.')[0] # example: 57114.html
    for url in urls:
        if cid == url[len('https://www.tcgplayer.com/product/'):].split('/')[0] :
            malformed_urls.append(url)
            continue
print(len(malformed_urls))

0


In [49]:
redo_results = []
for index, redo_url in enumerate(malformed_urls):
    product_id = redo_url[len('https://www.tcgplayer.com/product/'):].split('/')[0]
    redo_results.append(await get_url(redo_url, f'{output_dir}/{product_id}', overwrite=True))
    print(f'Completed {round((index+1)/(len(malformed_urls)), 4)*100}%{" "*100}', end='\r')

In [51]:
# run again here if done processing malformed
malformed_cards_redo = []
for index, filepath in enumerate([card[0] for card in malformed_cards]) :
    print(f'Processing {filepath} ({round(((index+1)/len(malformed_cards)), 4)*100}%){" "*50}', end='\r')
    # open HTML file
    with open(f'{output_dir}/{filepath}', 'r') as f:
        card_webpage = f.read()

    # extract and save
    try:
        card_extract = card_webpage_extract(card_webpage)
    except Exception as e:
        print(f'\n{filepath}\n{e}')
        malformed_cards_redo.append((filepath, e))
    
    card_data.append(card_extract)

In [53]:
import pandas as pd
card_df = pd.DataFrame(card_data)

In [59]:
card_df

,name,number,rarity,price,quantity,quantity_sellers,price_change_3m,index,asof
0,Gouki Guts - 2021 Tin of Ancient Battles (MP21),MP21-EN046,Common,$0.05,2314,212,(-14.29%),249048,1.723611e+09
1,Branded Disciple - Battle of Chaos (BACH),BACH-EN053,Common,$0.08,Market,Market,(+14.29%),262318,1.723601e+09
2,Chimeratech Overdragon - Star Pack 2014,SP14-EN043,Common,-,Market,Market,None,79857,1.723916e+09
3,Caius the Shadow Monarch - Gold Series 2009 (G...,GLD2-EN033,Ultra Rare,$4.24,Market,Market,(-5.05%),32106,1.724124e+09
4,De-Fusion - Duelist Pack 4: Zane Truesdale (DP04),DP04-EN017,Common,$0.23,335,80,None,26515,1.724172e+09
...,...,...,...,...,...,...,...,...,...
39894,Pazuzule - 2022 Tin of the Pharaoh's Gods (MP22),MP22-EN173,Common,$0.06,3583,253,(+25.00%),285022,1.723572e+09
39895,Steelswarm Sting - Duel Terminal 6 (DT06),DT06-EN030,Duel Terminal Rare Parallel Rare,-,217,26,None,81461,1.723863e+09
39896,Protector with Eyes of Blue - Legendary Decks ...,LDK2-ENK07,Common,$0.13,Market,Market,(+30.00%),543085,1.723444e+09
39897,One for One - Structure Deck: Pendulum Dominat...,SDPD-EN028,Common,$0.55,362,67,(-16.92%),127050,1.723787e+09


In [76]:
card_df[card_df['quantity'] == 'Market']

,name,number,rarity,price,quantity,quantity_sellers,price_change_3m,index,asof
1,Branded Disciple - Battle of Chaos (BACH),BACH-EN053,Common,$0.08,Market,Market,(+14.29%),262318,1.723601e+09
2,Chimeratech Overdragon - Star Pack 2014,SP14-EN043,Common,-,Market,Market,None,79857,1.723916e+09
3,Caius the Shadow Monarch - Gold Series 2009 (G...,GLD2-EN033,Ultra Rare,$4.24,Market,Market,(-5.05%),32106,1.724124e+09
5,Witch of the Black Forest - Structure Deck: Sp...,SDCH-EN016,Common,$0.36,Market,Market,(-7.89%),224981,1.724728e+09
6,Marauding Captain - Structure Deck: Warrior's ...,SD5-EN009,Common,$0.06,Market,Market,None,24528,1.724189e+09
...,...,...,...,...,...,...,...,...,...
39889,Number S0: Utopic ZEXAL - Maximum Crisis (MACR),MACR-ENSE2,Super Rare,$0.38,Market,Market,(-17.39%),133443,1.723779e+09
39892,Raidraptor - Pain Lanius - 2017 Mega-Tins Mega...,MP17-EN007,Common,$1.19,Market,Market,(-6.25%),141286,1.723775e+09
39893,Possessed Dark Soul - Legacy of Darkness (Worl...,LOD-EN004,Common,$0.25,Market,Market,None,476553,1.723558e+09
39896,Protector with Eyes of Blue - Legendary Decks ...,LDK2-ENK07,Common,$0.13,Market,Market,(+30.00%),543085,1.723444e+09


In [92]:
card_df2 = card_df[card_df['quantity'] != 'Market']
card_df2[card_df2['quantity'] == '']

,name,number,rarity,price,quantity,quantity_sellers,price_change_3m,index,asof
2933,Fire Formation - Gyokkou - Maximum Gold (MAGO),MAGO-EN073,Rare,,,,None,227499,1.723652e+09
7703,Retro Pack (2020 Date Reprint) Booster Box (Re...,None,None,,,,None,565623,1.724949e+09


In [57]:
card_df.to_csv('ygo_card_data_202408300224.csv')

### Sales Data
Now that we have saved out our final card data, we need to still reference the sales data in a separate database. The code utilizes the JSON output (e.g. 57114_sales.json) for each card to combine and output.

In [94]:
import os

sales_data = []
malformed = []
for index, filepath in enumerate(output_filepaths) :
    if filepath.split('.')[-1] == 'json' :
        print(f'Processing {filepath} ({round((index+1)/(len(output_filepaths)),4)*100} %){" "*50}', end='\r')
        full_path = f'{output_dir}/{filepath}'
        with open(full_path) as f:
            try:
                card_content = f.read()
                card_sales = json.loads(card_content)
                card_id = filepath.split('_')[0]

                # include fields like card id and modified time
                card_sales['id'] = card_id
                # we can get when this data was originally queried or valid by getting the file modified time
                card_sales['valid_time'] = os.path.getmtime(full_path)
                sales_data.append(card_sales)
            except Exception as e:
                print(f'{filepath} malformed', end='\r')
                malformed.append((filepath, e))

print(f'There were a total of {len(malformed)} malformed objects')

There were a total of 0 malformed objects                                                             


In [96]:
malformed[:1]

[]

In [98]:
malformed_urls = []
for redo in malformed:
    cid = redo[0].split('_')[0]
    for url in urls:
        if cid == url[len('https://www.tcgplayer.com/product/'):].split('/')[0] :
            malformed_urls.append(url)
            continue
print(len(malformed_urls))

0


In [99]:
redo_results = []
for index, redo_url in enumerate(malformed_urls):
    product_id = redo_url[len('https://www.tcgplayer.com/product/'):].split('/')[0]
    redo_results.append(await get_url(redo_url, f'{output_dir}/{product_id}', overwrite=True))
    print(f'Completed {round((index+1)/(len(malformed_urls)), 4)*100}%{" "*100}', end='\r')

In [100]:
import pandas as pd
sales_df = pd.DataFrame(sales_data)

In [101]:
sales_df

,previousPage,nextPage,resultCount,totalResults,data,id,valid_time
0,,,20,20,"[{'condition': 'Lightly Played', 'variant': 'U...",22100,1.724268e+09
1,,,0,0,[],213056,1.723675e+09
2,,,4,4,"[{'condition': 'Lightly Played', 'variant': '1...",32877,1.724121e+09
3,,Yes,25,823,"[{'condition': 'Near Mint', 'variant': '1st Ed...",199578,1.723687e+09
4,,Yes,25,54,"[{'condition': 'Near Mint', 'variant': '1st Ed...",256318,1.723604e+09
...,...,...,...,...,...,...,...
39861,,,20,20,"[{'condition': 'Lightly Played', 'variant': '1...",173586,1.723740e+09
39862,,Yes,25,308,"[{'condition': 'Moderately Played', 'variant':...",91001,1.723857e+09
39863,,Yes,25,67,"[{'condition': 'Near Mint', 'variant': '1st Ed...",106016,1.723824e+09
39864,,Yes,25,70,"[{'condition': 'Near Mint', 'variant': '1st Ed...",220906,1.723659e+09


In [71]:
# optional check for duplicates, this may take a while
sales_data_2 = [data for data in sales_data if len(data['data']) > 0]
sales_df_2 = pd.DataFrame(sales_data_2)
# perform a check for duplicated data
sales_df_3 = sales_df_2[sales_df_2.duplicated(subset=['data'])]

In [72]:
sales_df_3

,previousPage,nextPage,resultCount,totalResults,data,id,valid_time
33058,,,7,7,"[{'condition': 'Near Mint', 'variant': 'Unlimi...",285003,1.724633e+09


In [90]:
# redo these again
duplicate_urls = []
for cid in sales_df_3['id']:
    for url in urls:
        if cid == url[len('https://www.tcgplayer.com/product/'):].split('/')[0] :
            duplicate_urls.append(url)
            continue
print(f'Found {len(duplicate_urls)} duplicate urls.')

for index, redo_url in enumerate(duplicate_urls):
    product_id = redo_url[len('https://www.tcgplayer.com/product/'):].split('/')[0]
    redo_results.append(await get_url(redo_url, f'{output_dir}/{product_id}', overwrite=True))
    print(f'Completed {round((index+1)/(len(duplicate_urls)), 4)*100}%{" "*100}', end='\r')

Found 73 duplicate urls.



In [106]:
exploded_sales_data = sales_df.explode('data')
exploded_sales_data

,previousPage,nextPage,resultCount,totalResults,data,id,valid_time
0,,,20,20,"{'condition': 'Lightly Played', 'variant': 'Un...",22100,1.724268e+09
0,,,20,20,"{'condition': 'Moderately Played', 'variant': ...",22100,1.724268e+09
0,,,20,20,"{'condition': 'Near Mint', 'variant': 'Unlimit...",22100,1.724268e+09
0,,,20,20,"{'condition': 'Lightly Played', 'variant': 'Un...",22100,1.724268e+09
0,,,20,20,"{'condition': 'Near Mint', 'variant': '1st Edi...",22100,1.724268e+09
...,...,...,...,...,...,...,...
39865,,Yes,25,48,"{'condition': 'Lightly Played', 'variant': '1s...",116961,1.723815e+09
39865,,Yes,25,48,"{'condition': 'Moderately Played', 'variant': ...",116961,1.723815e+09
39865,,Yes,25,48,"{'condition': 'Near Mint', 'variant': '1st Edi...",116961,1.723815e+09
39865,,Yes,25,48,"{'condition': 'Near Mint', 'variant': '1st Edi...",116961,1.723815e+09


In [107]:
sales_df_flat = pd.concat([exploded_sales_data[["id", "valid_time"]].reset_index(drop=True), pd.json_normalize(exploded_sales_data["data"])], axis=1)

In [108]:
sales_df_flat.head()

,id,valid_time,condition,variant,language,quantity,title,listingType,customListingId,purchasePrice,shippingPrice,orderDate
0,22100,1.724268e+09,Lightly Played,Unlimited,English,1.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.25,1.27,2024-08-18T20:14:42.113+00:00
1,22100,1.724268e+09,Moderately Played,Unlimited,English,1.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.04,0.95,2024-08-13T21:02:28.257+00:00
2,22100,1.724268e+09,Near Mint,Unlimited,English,5.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.11,0.00,2024-08-13T00:56:53.23+00:00
3,22100,1.724268e+09,Lightly Played,Unlimited,English,5.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,0.09,0.00,2024-08-13T00:56:53.23+00:00
4,22100,1.724268e+09,Near Mint,1st Edition,English,1.0,Insect Soldiers of the Sky,ListingWithoutPhotos,0,1.24,0.00,2024-08-11T15:17:49.81+00:00


In [115]:
sales_df_flat.to_csv('ygo_sales_data_202408311524.csv')